In [2]:
from google.colab import drive
drive.mount('/content/drive')

KeyError: 'TBE_EPHEM_CREDS_ADDR'

In [2]:
import pandas as pd

# Specify the path to your CSV file
file_path = '/content/drive/MyDrive/BioCreativeVIII/BioREx/datas/dataset_train_novel/train_novel.csv'

# Read the CSV file into a Pandas DataFrame
dataset_train_novel = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
dataset_train_novel.head()


,Unnamed: 0,doc_id,relation,id_identifier_1,id_identifier_2,novel_cls,entity_1,identifier_1,full_sentence_1,entity_2,identifier_2,full_sentence_2
0,0,BC8_BioRED_Task1_Doc0,Positive_Correlation,c|DEL|-1607|G,C535531,Novel,"guanine insertion/deletion (G/D), the -1607 pr...",SequenceVariant,"A SNP for guanine insertion/deletion (G/D), t...",lumbar disc disease,Disease,Association between promoter -1607 polymorphis...
1,1,BC8_BioRED_Task1_Doc0,Association,D055959,c|DEL|-1607|G,Novel,DDD,Disease,Hence it is a good candidate for genetic stud...,"guanine insertion/deletion (G/D), the -1607 pr...",SequenceVariant,"A SNP for guanine insertion/deletion (G/D), t..."
2,2,BC8_BioRED_Task1_Doc0,Positive_Correlation,c|Allele|D|-1607,D055959,Novel,D allele for the -1607,SequenceVariant,CONCLUSION: We demonstrated that individuals ...,DDD,Disease,Hence it is a good candidate for genetic stud...
3,3,BC8_BioRED_Task1_Doc0,Association,4312,D055959,Novel,MMP1,Gene,Association between promoter -1607 polymorphis...,DDD,Disease,Hence it is a good candidate for genetic stud...
4,4,BC8_BioRED_Task1_Doc0,Association,C535531,4312,Novel,lumbar disc disease,Disease,Association between promoter -1607 polymorphis...,MMP1,Gene,Association between promoter -1607 polymorphis...


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
# Define the BERT tokenizer and model
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import os
from datetime import datetime
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
name_bert = "alvaroalon2/biobert_diseases_ner"
tokenizer = BertTokenizer.from_pretrained(name_bert)

KeyboardInterrupt: 

In [6]:
path_data_train = "/content/drive/MyDrive/BioCreativeVIII/BioREx/dataset_train_novel/train_novel.csv"
train_df = pd.read_csv(path_data_train, sep=',', encoding='utf-8')
le = LabelEncoder()
class_mapping = {
    'No': 0,
    'Novel': 1
}
# Fit and transform the 'pair type' column using the custom mapping
train_df['novel_cls'] = le.fit_transform(train_df['novel_cls'].map(class_mapping))
# Use train_test_split with stratify
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['novel_cls'], random_state=42)

In [8]:
# Tokenize and encode the sentences
class MyDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence1 = self.data.iloc[index]['full_sentence_1']
        sentence2 = self.data.iloc[index]['full_sentence_2']
        e1 = self.data.iloc[index]['entity_1']
        e2 = self.data.iloc[index]['entity_2']
        identifier1 = self.data.iloc[index]['identifier_1']
        identifier2 = self.data.iloc[index]['identifier_2']
        relation = self.data.iloc[index]['relation']
        

        novel_cls = self.data.iloc[index]['novel_cls']
        # input_text = f"{e1} [SEP] {e2}"
        input_text = f"{e1} - {identifier1} - {sentence1} [SEP] {e2} - {identifier2} - {sentence2} [SEP] {relation}"

        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': input_text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(novel_cls, dtype=torch.long)
        }

In [9]:
# create an instance of the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(name_bert)
# set the batch size
batch_size = 400
num_epochs = 150  # Adjust as needed

# set the maximum sequence length
max_len = 100

# create data loaders for the training and validation sets
train_dataset = MyDataset(train_df, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = MyDataset(val_df, tokenizer, max_len)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
class BertSentimentClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BertSentimentClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 256)
        self.dropout2 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, num_classes)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout1(pooled_output)
        output = self.fc1(pooled_output)
        output = self.dropout2(output)
        logits = self.fc2(output)

        return logits

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
num_classes = train_df["novel_cls"].nunique()
model = BertSentimentClassifier(name_bert, num_classes)
criterion = nn.CrossEntropyLoss()

In [ ]:
import torch.nn.functional as F

def focal_loss(outputs, labels, gamma=2.0, alpha=1.0):
    ce_loss = F.cross_entropy(outputs, labels, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = (alpha * (1 - pt) ** gamma * ce_loss).mean()
    return focal_loss

def train(model, data_loader, optimizer, criterion, device):
    # set the model to train mode
    model.train()

    # initialize the loss, accuracy, precision, recall, and f1_score variables
    total_loss, total_accuracy = 0, 0
    total_precision, total_recall, total_f1 = 0, 0, 0

    # iterate over the data loader
    for data in tqdm(data_loader):
        # move the inputs to the device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # zero the gradient
        optimizer.zero_grad()

        # get the model's predictions
        outputs = model(input_ids, attention_mask)

        # get the focal loss
        loss = focal_loss(outputs, labels)
        total_loss += loss.item()

        # accuracy calculation
        logits = outputs.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        total_accuracy += flat_accuracy(logits, label_ids)

        # calculate precision, recall, and f1 score
        total_precision += precision_score(label_ids, np.argmax(logits, axis=1), average='weighted')
        total_recall += recall_score(label_ids, np.argmax(logits, axis=1), average='weighted')
        total_f1 += f1_score(label_ids, np.argmax(logits, axis=1), average='weighted')

        # perform backpropagation and optimization
        loss.backward()
        optimizer.step()

    # calculate the average loss, accuracy, precision, recall, and f1 score
    avg_loss = total_loss / len(data_loader)
    # avg_accuracy = total_accuracy / len(data_loader)
    avg_precision = total_precision / len(data_loader)
    avg_recall = total_recall / len(data_loader)
    avg_f1 = total_f1 / len(data_loader)

    return avg_loss, avg_precision, avg_recall, avg_f1

In [ ]:
def evaluate(model, data_loader, criterion, device):
    # set the model to eval mode
    model.eval()

    # initialize the loss, accuracy, precision, recall, and f1_score variables
    total_loss = 0
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0

    # iterate over the data loader
    for data in tqdm(data_loader):
        # move the inputs to the device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # disable gradient computation
        with torch.no_grad():
            # get the model's predictions
            outputs = model(input_ids, attention_mask)

            # get the loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # calculate the number of correct predictions
            logits = outputs.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            total_accuracy += flat_accuracy(logits, label_ids)

            # calculate precision, recall, and f1 score
            total_precision += precision_score(label_ids, np.argmax(logits, axis=1), average='weighted')
            total_recall += recall_score(label_ids, np.argmax(logits, axis=1), average='weighted')
            total_f1 += f1_score(label_ids, np.argmax(logits, axis=1), average='weighted')

    # calculate the average loss, accuracy, precision, recall, and f1 score
    avg_loss = total_loss / len(data_loader)
    # accuracy = total_accuracy / len(data_loader)
    precision = total_precision / len(data_loader)
    recall = total_recall / len(data_loader)
    f1 = total_f1 / len(data_loader)

    return avg_loss, precision, recall, f1

In [ ]:
# Function to calculate F1 score, Precision, Recall
def calculate_metrics(predictions, labels):
    predictions = torch.argmax(predictions, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()

    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')

    return f1, precision, recall

In [ ]:
import warnings

# ...

# Ignore warnings
warnings.filterwarnings("ignore")

# Create a folder for logs if it doesn't exist
logs_folder = "logs"
os.makedirs(logs_folder, exist_ok=True)

# Generate a filename based on the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"{logs_folder}/training_log_BERT2_{current_time}.txt"



# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set the learning rate
lr = 2e-5

# set the number of epochs


# set the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# move the model to the device
model.to(device)

# lists to store metrics
train_losses = []
train_precisions = []
train_recalls = []
train_f1_scores = []


val_losses = []
val_precisions = []
val_recalls = []
val_f1_scores = []

best_test_loss = float('inf')
best_epoch = 0

# iterate over the epochs
# Set up logging
with open(log_filename, 'a') as log_file:
  for epoch in range(num_epochs):
      start_time = datetime.now()
      print(f'Epoch {epoch + 1}/{num_epochs}')

      # train the model
      train_loss, train_precision, train_recall, train_f1 = train(model, train_loader, optimizer, criterion, device)

      # evaluate the model on the validation set
      val_loss, val_precision, val_recall, val_f1 = evaluate(model, val_loader, criterion, device)


      # print the detailed metrics
      print(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}')
      print(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')

      # log the metrics
      log_file.write(f'Epoch {epoch + 1}/{num_epochs}\n')
      log_file.write(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}\n')
      log_file.write(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')
      log_file.flush()

      # Save the best model based on validation loss
      if val_loss < best_test_loss:
          best_test_loss = val_loss
          best_epoch = epoch + 1
          torch.save(model.state_dict(), f'best_model_BERT2.pt')

      # Save the metrics
      train_losses.append(train_loss)
      train_precisions.append(train_precision)
      train_recalls.append(train_recall)
      train_f1_scores.append(train_f1)

      val_losses.append(val_loss)
      val_precisions.append(val_precision)
      val_recalls.append(val_recall)
      val_f1_scores.append(val_f1)

      # Calculate and print time remaining
      elapsed_time = datetime.now() - start_time
      remaining_time = (num_epochs - epoch - 1) * elapsed_time
      print(f'Time Elapsed: {elapsed_time}, Time Remaining: {remaining_time} \n')
      torch.save(model.state_dict(), f'final_model_BERT2.pt')

# Print the best epoch
print(f"Best Model: Epoch {best_epoch}, Best Validation Loss: {best_test_loss:.4f}")

print(f"Path log file: ", log_filename)
